In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import datetime

In [2]:
encargos = pd.read_csv('../../encargos.csv')
viviendas = pd.read_csv('../../Viviendas.csv')
clima = pd.read_csv('../../clean_clima.csv')
intensitats = pd.read_csv('../../Probabilities.csv',index_col=0)
intensitats.index = pd.to_datetime(intensitats.index)
intensitats.index = [ x.date() for x in intensitats.index]

In [4]:
nameCols = encargos.columns.append(viviendas.columns[1:])
encargosColumns = encargos.columns
print(encargos.columns)
print(viviendas.columns)

Index(['TimOcurrencia', 'DesCPOcurrencia', 'NumEncargos', 'AccidenteDeFumador',
       'AccidenteDomestico', 'ActosVandalicos', 'AsentamientosHundimientos',
       'AveriaDeLaInstalacion', 'AveriaInternaDelAparato', 'CaidaDeObjetos',
       'ConsecuenciasDeErrorDeDiseno', 'Cortocircuito', 'DanosPorAgua',
       'DefectosDeMateriales', 'DefectoEstudioGeotecnico',
       'DefectoFalloProyectoCalculo', 'ElectricidadEstatica', 'Emboces',
       'Expolacion', 'CaidaDeRayo', 'Filtraciones',
       'FugasInstalacionesDeExtincion', 'GolpeDeAire', 'GolpeFortuito',
       'GoterasProcedentesDeLocales', 'Humo', 'Hurto', 'IgnicionEspontanea',
       'ImpactoVehiculos', 'IncendiosDeOrigenElectrico', 'IntentoDeRobo',
       'InundacionDesbordamientoDesviacion', 'LlamasAbiertas', 'Lluvia',
       'AveriasAparatosElectricos', 'ManipulacionMaquinariaBienesAsegurados',
       'ManoDeObraDefectuosa', 'MaterialesRecalentados', 'MontajeAnomalo',
       'Nieve', 'NinoJugando', 'OlvidoCierreGrifos', 'OndasSo

In [5]:
encargos['TimOcurrencia'] = pd.to_datetime(encargos['TimOcurrencia'])
encargos['TimOcurrencia'] = encargos['TimOcurrencia'].dt.date
clima['DATA_LECTURA'] = pd.to_datetime(clima['DATA_LECTURA'])
clima['DATA_LECTURA'] = clima['DATA_LECTURA'].dt.date
print(encargos['TimOcurrencia'])
encargos.sort_values(by=['TimOcurrencia'], inplace=True, ascending=True)
viviendas.sort_values(by=['DesCPOcurrencia'], inplace=True, ascending=True)



0         2017-03-04
1         2017-01-05
2         2018-09-26
3         2019-01-18
4         2019-01-09
             ...    
227216    2017-09-26
227217    2017-10-31
227218    2018-05-29
227219    2019-02-14
227220    2018-09-10
Name: TimOcurrencia, Length: 227221, dtype: object


In [6]:
valuesEncargos = encargos['TimOcurrencia'].unique()
valuesViviendas = viviendas['DesCPOcurrencia'].unique()
allCPs = clima['CP'].unique()



In [7]:
def isValueInArray(CP,array):
  i = 0
  for value in array:
      if CP == value:
          return True, i
      i += 1
  return False, -1

In [8]:
data = []
numCols = encargos.columns.size + viviendas.columns.size - 1
indexDia = 0

for dia in pd.date_range(start="2017-01-01",end="2019-12-31").to_pydatetime().tolist():
    dia = dia.date()
    createDay = False
    if dia in valuesEncargos:
        subarray = encargos.loc[encargos['TimOcurrencia'] == dia]
        subarray.sort_values(by=['DesCPOcurrencia'], inplace=True, ascending=True)
    else:
        subarray = encargos.iloc[0]
        createDay = True

    for CP in allCPs:
        CP = np.int64(CP)
        isVal = False
        if not createDay:
            isVal, index = isValueInArray(CP,subarray['DesCPOcurrencia'])
        auxiliarRow = []
        if isVal:
            auxiliarRow = subarray.iloc[index]
        else:
            auxiliarRow = subarray.iloc[0]
            for col in encargosColumns:
                if col == 'TimOcurrencia':
                    auxiliarRow[col] = dia
                elif col == 'DesCPOcurrencia':
                    auxiliarRow[col] = CP
                else:
                    auxiliarRow[col] = 0

        locatedCP = viviendas.loc[ viviendas['DesCPOcurrencia'] == CP ].iloc[0]
        auxiliarRow['AnoConstruccion'] = np.int64(locatedCP['AnoConstruccion'])
        auxiliarRow['Clase_Alta'] = np.int64(locatedCP['Clase_Alta'])
        auxiliarRow['Clase_MediaAlta'] = np.int64(locatedCP['Clase_MediaAlta'])
        auxiliarRow['Clase_Media'] = np.int64(locatedCP['Clase_Media'])
        auxiliarRow['Clase_MediaSencilla'] = np.int64(locatedCP['Clase_MediaSencilla'])
        auxiliarRow['Clase_Sencilla'] = np.int64(locatedCP['Clase_Sencilla'])

        locatedClima = clima.loc[ clima['DATA_LECTURA'] == dia ]
        locatedClima = locatedClima.loc[ locatedClima['CP'] == CP ]
        auxiliarRow['Temperatura_max'] = np.float64(locatedClima['Temperatura_max'])
        auxiliarRow['Temperatura_min'] = np.float64(locatedClima['Temperatura_min'])
        auxiliarRow['Humitat_max'] = np.float64(locatedClima['Humitat_max'])
        auxiliarRow['Humitat_min'] = np.float64(locatedClima['Humitat_min'])
        auxiliarRow['Precipitacio_max'] = np.float64(locatedClima['Precipitacio_max'])
        auxiliarRow['Precipitacio_min'] = np.float64(locatedClima['Precipitacio_min'])
        auxiliarRow['IrrSolar_max'] = np.float64(locatedClima['IrrSolar_max'])
        auxiliarRow['IrrSolar_min'] = np.float64(locatedClima['IrrSolar_min'])
        auxiliarRow['Neu_max'] = np.float64(locatedClima['Neu_max'])
        auxiliarRow['Neu_min'] = np.float64(locatedClima['Neu_min'])
        auxiliarRow['Vent_max'] = np.float64(locatedClima['Vent_max'])
        auxiliarRow['Vent_min'] = np.float64(locatedClima['Vent_min'])

        auxiliarRow['ProbabilitatManifestacio'] = np.float64(intensitats.at[dia,'Pm'])
        auxiliarRow['ProbabilitatFesta'] = np.float64(intensitats.at[dia,'Pf'])

        data.append(auxiliarRow)

    indexDia += 1
    print(indexDia/1095*100)


0.091324200913242
0.182648401826484
0.273972602739726
0.365296803652968
0.45662100456621
0.547945205479452
0.6392694063926941
0.730593607305936
0.821917808219178
0.91324200913242
1.004566210045662
1.095890410958904
1.187214611872146
1.2785388127853883
1.36986301369863
1.461187214611872
1.552511415525114
1.643835616438356
1.7351598173515983
1.82648401826484
1.9178082191780823
2.009132420091324
2.1004566210045663
2.191780821917808
2.28310502283105
2.374429223744292
2.4657534246575343
2.5570776255707766
2.6484018264840183
2.73972602739726
2.8310502283105023
2.922374429223744
3.0136986301369864
3.105022831050228
3.1963470319634704
3.287671232876712
3.3789954337899544
3.4703196347031966
3.5616438356164384
3.65296803652968
3.744292237442922
3.8356164383561646
3.9269406392694064
4.018264840182648
4.10958904109589
4.200913242009133
4.292237442922374
4.383561643835616
4.474885844748858
4.5662100456621
4.657534246575342
4.748858447488584
4.840182648401827
4.931506849315069
5.0228310502283104
5.1

In [9]:
pd.DataFrame(data).to_csv('../clean_data.csv', index=False)